In [718]:
# load libraries

import pandas as pd
import numpy as np
import re
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.set_option('display.max_colwidth',100)

In [719]:
def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5

In [720]:
def find_nan_cols(train_df):
    nan_cols = []
    for col in train_df.columns:
        if(sum(pd.isnull(train_df[col]))>0):
            nan_cols.append(col)
    return nan_cols        

In [721]:
#load data
demographics = pd.read_csv('train/demographics.csv')
event_calender = pd.read_csv('train/event_calendar.csv')
historical_volume = pd.read_csv('train/historical_volume.csv')
industry_soda_sales = pd.read_csv('train/industry_soda_sales.csv')
industry_volume = pd.read_csv('train/industry_volume.csv')
price_sales_promotion = pd.read_csv('train/price_sales_promotion.csv')
weather = pd.read_csv('train/weather.csv')

In [722]:
sku_recommendation = pd.read_csv('test/sku_recommendation.csv')
volume_forecast = pd.read_csv('test/volume_forecast.csv')

In [723]:
historical_volume.head(2)

,Agency,SKU,YearMonth,Volume
0,Agency_22,SKU_01,201301,52.272
1,Agency_22,SKU_02,201301,110.700


In [724]:
sku_agg = historical_volume.groupby(['SKU']). \
    agg({'Volume' : [np.min,np.mean,np.max]}).reset_index()
sku_agg.columns = ['SKU', 'min_Volume_SKU', 'mean_Volume_SKU', 'max_Volume_SKU']

In [725]:
sku_agg.head()

,SKU,min_Volume_SKU,mean_Volume_SKU,max_Volume_SKU
0,SKU_01,0.0,3194.051328,22126.6050
1,SKU_02,0.0,2208.376728,17624.5200
2,SKU_03,0.0,1845.716049,22526.6100
3,SKU_04,0.0,1188.229987,8665.6725
4,SKU_05,0.0,1259.238693,10583.6325


In [726]:
agency_agg = historical_volume.groupby(['Agency']). \
    agg({'Volume' : [np.min,np.mean,np.max]}).reset_index()
agency_agg.columns = ['Agency', 'min_Volume_Agency', 'mean_Volume_Agency', 'max_Volume_Agency']

In [727]:
agency_agg.head()

,Agency,min_Volume_Agency,mean_Volume_Agency,max_Volume_Agency
0,Agency_01,0.0,70.175700,514.7145
1,Agency_02,0.0,4033.935444,22526.6100
2,Agency_03,0.0,2610.920059,13354.3650
3,Agency_04,0.0,146.252602,938.6370
4,Agency_05,0.0,2174.659879,22126.6050


In [728]:
agency_sku_agg = historical_volume.groupby(['Agency', 'SKU']). \
    agg({'Volume' : [np.min,np.mean,np.max]}).reset_index()
agency_sku_agg.columns = ['Agency', 'SKU', 'min_Volume_SKU_Agency', 'mean_Volume_SKU_Agency', 'max_Volume_SKU_Agency']

In [729]:
agency_sku_agg.head()

,Agency,SKU,min_Volume_SKU_Agency,mean_Volume_SKU_Agency,max_Volume_SKU_Agency
0,Agency_01,SKU_01,4.3200,83.523600,180.7920
1,Agency_01,SKU_02,17.1720,66.898800,171.3960
2,Agency_01,SKU_03,0.0000,40.777465,106.6806
3,Agency_01,SKU_04,15.2082,191.208123,514.7145
4,Agency_01,SKU_05,11.4750,20.484120,40.3209


In [730]:
train_df = pd.merge(historical_volume, demographics, on='Agency', how='left')
train_df = pd.merge(train_df, event_calender, on='YearMonth', how='left')
train_df = pd.merge(train_df, agency_agg, on='Agency', how='left')
train_df = pd.merge(train_df, sku_agg, on='SKU', how='left')
# train_df = pd.merge(train_df, agency_sku_agg, on=['Agency', 'SKU'], how='left')

# train_df = pd.merge(train_df, industry_soda_sales, on='YearMonth', how='left')
# train_df = pd.merge(train_df, industry_volume, on='YearMonth', how='left')

In [731]:
train_df.head()

,Agency,SKU,YearMonth,Volume,Avg_Population_2017,Avg_Yearly_Household_Income_2017,Easter Day,Good Friday,New Year,Christmas,...,FIFA U-17 World Cup,Football Gold Cup,Beer Capital,Music Fest,min_Volume_Agency,mean_Volume_Agency,max_Volume_Agency,min_Volume_SKU,mean_Volume_SKU,max_Volume_SKU
0,Agency_22,SKU_01,201301,52.2720,48151,132110,0,0,1,0,...,0,0,0,0,23.1318,143.650922,425.0628,0.0,3194.051328,22126.6050
1,Agency_22,SKU_02,201301,110.7000,48151,132110,0,0,1,0,...,0,0,0,0,23.1318,143.650922,425.0628,0.0,2208.376728,17624.5200
2,Agency_58,SKU_23,201301,0.0000,1620969,105857,0,0,1,0,...,0,0,0,0,0.0000,1771.901852,10246.0650,0.0,0.884812,18.9900
3,Agency_48,SKU_07,201301,28.3200,1739969,210213,0,0,1,0,...,0,0,0,0,0.0000,1997.797814,13462.3950,0.0,33.140397,202.6650
4,Agency_22,SKU_05,201301,238.5387,48151,132110,0,0,1,0,...,0,0,0,0,23.1318,143.650922,425.0628,0.0,1259.238693,10583.6325


In [732]:
test_df = pd.DataFrame(data=volume_forecast)
test_df['YearMonth'] = 201801
# industry_soda_sales.loc[60] = [201801, 885614054]
# industry_volume.loc[60] = [201801, 560004016]

In [733]:
test_df = pd.merge(volume_forecast, demographics, on='Agency', how='left')
test_df = pd.merge(test_df, event_calender, on='YearMonth', how='left')
test_df = pd.merge(test_df, agency_agg, on='Agency', how='left')
test_df = pd.merge(test_df, sku_agg, on='SKU', how='left')

# test_df = pd.merge(test_df, agency_sku_agg, on=['Agency', 'SKU'], how='left')

In [734]:
test_df_copy = test_df.copy()

In [735]:
test_df_copy.head()

,Agency,SKU,Volume,YearMonth,Avg_Population_2017,Avg_Yearly_Household_Income_2017,Easter Day,Good Friday,New Year,Christmas,...,FIFA U-17 World Cup,Football Gold Cup,Beer Capital,Music Fest,min_Volume_Agency,mean_Volume_Agency,max_Volume_Agency,min_Volume_SKU,mean_Volume_SKU,max_Volume_SKU
0,Agency_23,SKU_06,NaN,201801,49101,90240,0,0,1,0,...,0,0,0,0,0.000,131.064589,942.65250,0.648,127.760400,937.44000
1,Agency_34,SKU_20,NaN,201801,12271,97222,0,0,1,0,...,0,0,0,0,0.000,3.623409,40.70430,0.000,4.329854,47.31795
2,Agency_24,SKU_02,NaN,201801,42122,93027,0,0,1,0,...,0,0,0,0,0.000,119.185946,776.70450,0.000,2208.376728,17624.52000
3,Agency_40,SKU_06,NaN,201801,143299,137912,0,0,1,0,...,0,0,0,0,0.000,49.002228,900.22350,0.648,127.760400,937.44000
4,Agency_42,SKU_32,NaN,201801,175794,110057,0,0,1,0,...,0,0,0,0,17.064,440.257746,1981.67025,0.000,25.091122,207.35550


In [736]:
test_df.shape

(1450, 24)

In [737]:
feat = ['Agency','SKU']

for x in feat:
    le = LabelEncoder()
    le.fit(list(test_df[x].values) + list(train_df[x].values))
    train_df[x] = le.transform(list(train_df[x]))
    test_df[x] = le.transform(list(test_df[x]))

In [738]:
test_df.drop('Volume', axis=1, inplace=True)

In [739]:
train_df.head()

,Agency,SKU,YearMonth,Volume,Avg_Population_2017,Avg_Yearly_Household_Income_2017,Easter Day,Good Friday,New Year,Christmas,...,FIFA U-17 World Cup,Football Gold Cup,Beer Capital,Music Fest,min_Volume_Agency,mean_Volume_Agency,max_Volume_Agency,min_Volume_SKU,mean_Volume_SKU,max_Volume_SKU
0,19,0,201301,52.2720,48151,132110,0,0,1,0,...,0,0,0,0,23.1318,143.650922,425.0628,0.0,3194.051328,22126.6050
1,19,1,201301,110.7000,48151,132110,0,0,1,0,...,0,0,0,0,23.1318,143.650922,425.0628,0.0,2208.376728,17624.5200
2,55,17,201301,0.0000,1620969,105857,0,0,1,0,...,0,0,0,0,0.0000,1771.901852,10246.0650,0.0,0.884812,18.9900
3,45,6,201301,28.3200,1739969,210213,0,0,1,0,...,0,0,0,0,0.0000,1997.797814,13462.3950,0.0,33.140397,202.6650
4,19,4,201301,238.5387,48151,132110,0,0,1,0,...,0,0,0,0,23.1318,143.650922,425.0628,0.0,1259.238693,10583.6325


In [740]:
train_df.columns

Index(['Agency', 'SKU', 'YearMonth', 'Volume', 'Avg_Population_2017',
       'Avg_Yearly_Household_Income_2017', 'Easter Day', 'Good Friday',
       'New Year', 'Christmas', 'Labor Day', 'Independence Day',
       'Revolution Day Memorial', 'Regional Games ', 'FIFA U-17 World Cup',
       'Football Gold Cup', 'Beer Capital', 'Music Fest', 'min_Volume_Agency',
       'mean_Volume_Agency', 'max_Volume_Agency', 'min_Volume_SKU',
       'mean_Volume_SKU', 'max_Volume_SKU'],
      dtype='object')

In [741]:
cols = ['Agency', 'SKU', 'YearMonth', 'Avg_Population_2017',
       'Avg_Yearly_Household_Income_2017', 'Easter Day', 'Good Friday',
       'New Year', 'Christmas', 'Labor Day', 'Independence Day',
       'Revolution Day Memorial', 'Regional Games ', 'FIFA U-17 World Cup',
       'Football Gold Cup', 'Beer Capital', 'Music Fest', 'min_Volume_Agency',
       'mean_Volume_Agency', 'max_Volume_Agency', 'min_Volume_SKU',
       'mean_Volume_SKU', 'max_Volume_SKU']

In [742]:
find_nan_cols(test_df)

[]

In [743]:
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_df[cols], 
                                                    train_df['Volume'], 
                                                    test_size = 0.25, 
                                                    random_state = 77)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 15750 samples.
Testing set has 5250 samples.


In [ ]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

clf_A = ExtraTreesRegressor(random_state=77, n_jobs=-1, max_depth=25, n_estimators=18, min_samples_split=6, 
                           max_leaf_nodes=10000)
clf_C = BaggingRegressor(random_state=77, n_estimators=250, n_jobs=-1)
clf_D = LGBMRegressor(n_jobs=-1, random_state=77, num_leaves=95)
#clf_E = RandomForestRegressor(random_state=77, n_jobs=-1, n_estimators=250)
clf_E = XGBRegressor(n_estimators=500, max_depth=8, learning_rate= 0.1, min_child_weight=5)


clf_A.fit(X_train, y_train)
#clf_C.fit(X_train, y_train)
#clf_D.fit(X_train, y_train)
clf_E.fit(X_train, y_train)

preds1 = clf_A.predict(X_test)
#preds3 = clf_C.predict(X_test)
#preds4 = clf_D.predict(X_test)
preds5 = clf_E.predict(X_test)

print('{} has precision: {}'.format(clf_A.__class__.__name__, RMSLE((y_test), (preds1))))
#print('{} has precision: {}'.format(clf_C.__class__.__name__, r2_score(y_test, preds3)))
#print('{} has precision: {}'.format(clf_D.__class__.__name__, r2_score(y_test, preds4)))
print('{} has precision: {}'.format(clf_E.__class__.__name__, RMSLE(y_test, preds5)))

In [ ]:
preds1 = clf_A.predict(test_df)
preds5 = clf_E.predict(test_df)

In [ ]:
preds = (preds1+preds5)/2

In [ ]:
preds = np.where(preds<0.0,0,preds)

In [ ]:
sub = pd.DataFrame()
sub = pd.concat([test_df_copy[['Agency', 'SKU']], pd.Series(preds)], axis=1)
sub = sub.rename(columns={0:'Volume'})
sub[['Volume']] = sub[['Volume']].apply(pd.to_numeric)
sub.to_csv('volume_forecast.csv', index=False)